In [1]:
import pandas as pd
import numpy as np

## Importamos la tabla de datos descargada del INE


In [6]:
df = pd.read_csv('pob-total-pc.csv', sep=';', index_col=0, skiprows=5, engine='python', skipfooter=2, encoding='UTF-8')

In [7]:
df

,1 de julio de 2019,1 de enero de 2019,1 de julio de 2018,1 de enero de 2018,1 de julio de 2017,1 de enero de 2017,1 de julio de 2016,1 de enero de 2016,1 de julio de 2015,1 de enero de 2015,...,1 de enero de 1975,1 de julio de 1974,1 de enero de 1974,1 de julio de 1973,1 de enero de 1973,1 de julio de 1972,1 de enero de 1972,1 de julio de 1971,1 de enero de 1971,Unnamed: 99
Ambos sexos,NaN,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
Total,NaN,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
Total Nacional,47.100.396,46.937.060,46.728.814,46.658.447,46.532.869,46.527.039,46.449.874,46.440.099,46.410.149,46.449.565,...,35.569.375,35.363.890,35.177.294,34.980.317,34.800.600,34.595.886,34.408.338,34.216.856,34.040.642,NaN
02 Albacete,389.584,389.528,389.322,390.337,390.304,391.574,392.036,392.958,393.664,394.928,...,341.203,341.120,341.223,341.191,341.184,340.941,340.896,340.750,340.709,NaN
03 Alicante/Alacant,1.873.295,1.862.780,1.848.500,1.845.470,1.838.940,1.842.572,1.839.982,1.842.446,1.839.975,1.842.174,...,1.011.877,999.864,988.791,977.012,965.894,954.254,943.452,932.507,921.985,NaN
04 Almería,710.808,706.871,701.831,701.069,700.704,700.041,698.033,696.186,693.634,691.155,...,390.688,388.759,387.553,385.658,384.280,382.386,380.880,379.087,377.637,NaN
01 Araba/Álava,328.571,327.682,326.419,325.739,324.569,323.812,322.706,321.804,321.077,320.663,...,223.028,220.074,217.047,214.139,211.227,208.303,205.372,202.665,199.776,NaN
33 Asturias,1.019.993,1.022.205,1.024.192,1.027.624,1.029.953,1.034.277,1.037.011,1.040.925,1.044.043,1.049.875,...,1.085.361,1.080.982,1.077.057,1.072.722,1.069.075,1.064.504,1.060.233,1.055.764,1.052.039,NaN
05 Ávila,158.966,159.260,159.251,160.032,160.519,161.749,162.611,163.650,164.505,165.741,...,200.348,201.686,203.195,204.548,205.980,207.306,208.727,210.144,211.556,NaN
06 Badajoz,671.079,672.493,673.221,674.926,676.136,678.473,680.350,682.572,683.822,686.092,...,680.857,683.708,686.680,689.111,692.036,694.133,696.679,699.062,701.675,NaN


In [ ]:
df = df.reset_index()

In [ ]:
df

#### Cambiamos el nombre de la columna 'Index' por 'Provincias' y limpiamos todos los espacios en blanco que tenga

In [ ]:
df.rename(columns={'index': 'provincias'}, inplace=True)

In [ ]:
df['provincias'] = df["provincias"].map(str.strip)

In [ ]:
df['provincias'].unique()

### Para extraer el sexo de las personas 

#### Creamos una nueva columna a partir de 'Provincias' en la que se encuentra también el sexo. 

Cambiamos todos los valores a vacío en 'in_sexo' excepto los de género. 
Después rellenamos los huecos con fillna

In [ ]:
# df['in_sexo'] = df['provincias']

In [ ]:
df['in_sexo'] = df['provincias'][df['provincias'].str.contains('Ambos sexos|Hombres|Mujeres')]

In [ ]:
df

In [ ]:
df['in_sexo'].fillna(method='ffill', inplace=True)

In [ ]:
df

#### No nos interesan las provincias ya que tenemos el total de cada grupo de edad bajo el apéndice 'Total Nacional'.

Creamos una lista y seleccionamos solo las provincias para después restarlas. 

In [ ]:
prov = df['provincias'].unique().tolist()[3:55]

In [ ]:
prov

In [ ]:
df.drop(0, inplace=True)

In [ ]:
df

#### Seleccionamos todos los valores que no estén en la lista de 'Provincias'

In [ ]:
df = df[-df['provincias'].isin(prov)]

In [ ]:
df

# Hacemos dos dataframes para restarlos después 

#### df_total es el dataframe con valores que no tienen la etiqueta 'Total Nacional'. Contiene todos los valores vacíos que usaremos para restar después.
#### dl_clean es una copia del dataframe original hasta este punto para guardarlo por si acaso. Para ello lo copiamos.

In [ ]:
df_total = df[df['provincias'] != 'Total Nacional']

In [ ]:
df_total

In [ ]:
df_clean = df.copy()

In [ ]:
df_clean

En el 'DataFrame Clean' eliminamos los valores 'Total Nacional' de la columna 'Provincias' para bajar una fila el nombre de los grupos de edad y que queden en la misma fila que las cifras

In [ ]:
df_clean['provincias'].replace('Total Nacional', np.nan, inplace=True)

In [ ]:
df_clean['provincias'].fillna(method='ffill', inplace=True)

In [ ]:
df_clean

Ahora restamos el DataFrame con los valores vacíos y el DataFrame que vamos a utilizar. Lo hacemos sumándolos y eliminando los valores duplicados, así nos queda limpio.

In [ ]:
result = pd.concat([df_clean, df_total]).drop_duplicates(keep=False)

In [ ]:
result

In [ ]:
result.set_index(['in_sexo', 'provincias'])

Cambiamos el nombre de la columna 'provincias' por 'edad'

In [ ]:
result = result.rename(columns={'provincias':'edad'})

#### Limpiamos las columnas para quitar todas las columnas que contengan la palabra 'julio'

Para ello creamos una lista con el nombre de todas las columnas y después, con un bucle, iteramos entre los valores de la lista para eliminar de esa misma lista todos los valores que contengan la palabra 'julio'

In [ ]:
col = result.columns.tolist()

In [ ]:
for i in col:
    if 'julio' in i:
        col.remove(i)
    else:
        continue

In [ ]:
col

#### También quitamos todos los valores menores de 1980

In [ ]:
col = col[:41]+col[-1:]

In [ ]:
col = col[:1]+col[2:]

In [ ]:
col

Aplicamos esas columnas seleccionadas a nuestro DataFrame bajo el nombre 'result'

In [ ]:
result = result[col]

In [ ]:
result

Cambiamos el nombre de las columnas para evitar confusiones cuando la mezclemos con la tabla de los suicidios. La llamamos 'Población X' donde 'X' es cada año. Para ello creamos una lista. 

In [ ]:
años = list()

In [ ]:
for i in result.columns[1:-1]:
    año = i.split()[4]
    año = 'Poblacion: ' + año
    años.append(año)

In [ ]:
columnas = col[:1]+años+col[-1:]

In [ ]:
columnas

Cambiamos el nombre de las columnas en base a la lista creada antes.

In [ ]:
result.columns = columnas

In [ ]:
result

## En la tabla de "Defunciones por causa", en la que se encuentran las cifras de los suicidios, los valores llegan hasta los '95 años y más'. Además se encuentran agrupados por grupos de 4 años y en este modelo los tenemos divididos por cada año. Habrá que sumarlos. 


En primer lugar debemos eliminar la columna '85 años y más'. El INE ofrece la población por cada año ('1 año', '2 años'), pero al final de la lista tenemos la opción de buscar '85 años y más'. En este caso lo vamos a eliminar porque nos quedamos con los grupos de edad por cada año.

In [ ]:
result = result[-result['edad'].str.contains('85 y más años')]

Convertimos a números los datos, el programa los interpretaba hasta ahora (debido al punto entre las cifra) como una cadena de texto. Eliminamos el punto y los convertimos a 'integer'.

In [ ]:
result['Poblacion: 2018'].str.split('.').str.join('')

In [ ]:
result.loc[:, 'Poblacion: 2018' : 'Poblacion: 1980'] = result.loc[:, 'Poblacion: 2018' : 'Poblacion: 1980'].apply(lambda x: x.str.split('.').str.join(''))

In [ ]:
pob = result.loc[:, 'Poblacion: 2018' : 'Poblacion: 1980'].apply(lambda x: pd.to_numeric(x))

In [ ]:
pob

In [ ]:
ind = result[['in_sexo', 'edad']]

In [ ]:
total = pd.merge(pob, ind, left_index=True, right_index=True)

In [ ]:
col = total.columns.tolist()

In [ ]:
col = col[-2:] + col[:-2]

In [ ]:
total = total[col]

In [ ]:
total

Al agruparlos por sexo y sumar todos los resultados nos devuelve una cifra que es el doble de lo esperado. Esto es normal ya que tenemos agrupados tanto los valores divididos por año pero también el total de cada sexo. Por eso devuelve el doble pero no es algo definitivo.

In [ ]:
total.groupby('in_sexo').sum()

#### La tabla con los datos de los suidicios viene por grupos de edad. 

Tenemos que crear nosotros grupos de igual valor para después poder compararlos con los suicidios. Para ello creamos un mapa para decirle al programa qué edad corresponde a cada grupo de edad.

In [ ]:
edades = {
'Total': 'Todas las edades',
'0 años' : 'Menos de 1 año',
'1 año': 'De 1 a 4 años',
'2 años': 'De 1 a 4 años',
'3 años': 'De 1 a 4 años',
'4 años': 'De 1 a 4 años',
'5 años': 'De 5 a 9 años',
'6 años': 'De 5 a 9 años',
'7 años': 'De 5 a 9 años',
'8 años': 'De 5 a 9 años',
'9 años': 'De 5 a 9 años',
'10 años': 'De 10 a 14 años',
'11 años': 'De 10 a 14 años',
'12 años': 'De 10 a 14 años',
'13 años': 'De 10 a 14 años',
'14 años': 'De 10 a 14 años',
'15 años': 'De 15 a 19 años',
'16 años': 'De 15 a 19 años',
'17 años': 'De 15 a 19 años',
'18 años': 'De 15 a 19 años',
'19 años': 'De 15 a 19 años',
'20 años': 'De 20 a 24 años',
'21 años': 'De 20 a 24 años',
'22 años': 'De 20 a 24 años',
'23 años': 'De 20 a 24 años',
'24 años': 'De 20 a 24 años',
'25 años': 'De 25 a 29 años',
'26 años': 'De 25 a 29 años',
'27 años': 'De 25 a 29 años',
'28 años': 'De 25 a 29 años',
'29 años': 'De 25 a 29 años',
'30 años': 'De 30 a 34 años',
'31 años': 'De 30 a 34 años',
'32 años': 'De 30 a 34 años',
'33 años': 'De 30 a 34 años',
'34 años': 'De 30 a 34 años',
'35 años': 'De 35 a 39 años',
'36 años': 'De 35 a 39 años',
'37 años': 'De 35 a 39 años',
'38 años': 'De 35 a 39 años',
'39 años': 'De 35 a 39 años',
'40 años': 'De 40 a 44 años',
'41 años': 'De 40 a 44 años',
'42 años': 'De 40 a 44 años',
'43 años': 'De 40 a 44 años',
'44 años': 'De 40 a 44 años',
'45 años': 'De 45 a 49 años',
'46 años': 'De 45 a 49 años',
'47 años': 'De 45 a 49 años',
'48 años': 'De 45 a 49 años',
'49 años': 'De 45 a 49 años',
'50 años': 'De 50 a 54 años',
'51 años': 'De 50 a 54 años',
'52 años': 'De 50 a 54 años',
'53 años': 'De 50 a 54 años',
'54 años': 'De 50 a 54 años',
'55 años': 'De 55 a 59 años',
'56 años': 'De 55 a 59 años',
'57 años': 'De 55 a 59 años',
'58 años': 'De 55 a 59 años',
'59 años': 'De 55 a 59 años',
'60 años': 'De 60 a 64 años',
'61 años': 'De 60 a 64 años',
'62 años': 'De 60 a 64 años',
'63 años': 'De 60 a 64 años',
'64 años': 'De 60 a 64 años',
'65 años': 'De 65 a 69 años',
'66 años': 'De 65 a 69 años',
'67 años': 'De 65 a 69 años',
'68 años': 'De 65 a 69 años',
'69 años': 'De 65 a 69 años',
'70 años': 'De 70 a 74 años',
'71 años': 'De 70 a 74 años',
'72 años': 'De 70 a 74 años',
'73 años': 'De 70 a 74 años',
'74 años': 'De 70 a 74 años',
'75 años': 'De 75 a 79 años',
'76 años': 'De 75 a 79 años',
'77 años': 'De 75 a 79 años',
'78 años': 'De 75 a 79 años',
'79 años': 'De 75 a 79 años',
'80 años': 'De 80 a 84 años',
'81 años': 'De 80 a 84 años',
'82 años': 'De 80 a 84 años',
'83 años': 'De 80 a 84 años',
'84 años': 'De 80 a 84 años',
'85 años': 'De 85 a 89 años',
'86 años': 'De 85 a 89 años',
'87 años': 'De 85 a 89 años',
'88 años': 'De 85 a 89 años',
'89 años': 'De 85 a 89 años',
'90 años': 'De 90 a 94 años',
'91 años': 'De 90 a 94 años',
'92 años': 'De 90 a 94 años',
'93 años': 'De 90 a 94 años',
'94 años': 'De 90 a 94 años',
'95 años': '95 y más años',
'96 años': '95 y más años',
'97 años': '95 y más años',
'98 años': '95 y más años',
'99 años': '95 y más años',
'100 y más años': '95 y más años'
}

Creamos una nueva columna (grupos de edad) aplicando el mapa en base a la columna 'edad'.

In [ ]:
total['gr_edad'] = total['edad'].map(edades)

Sumamos los valores de población agrupándolos por sexo y por grupos de edad

In [ ]:
casi = total.groupby(['in_sexo', 'gr_edad']).sum()

In [ ]:
casi = casi.reset_index()

In [ ]:
casi['in_sexo'][casi['in_sexo'] == 'Ambos sexos'] = 'Total'

In [ ]:
casi

In [ ]:
casi = casi.set_index(['in_sexo', 'gr_edad'])

### Pivotamos la tabla. Mantenemos como columnas el sexo y los grupos de edad y pivotamos las demás columnas que corresponen a los valores de población por año para tenerlo todo en un formato de columnas. 

#### Dicho formato corresponde al formato de la tabla "Defunciones por causa" en la que tenemos los valores de suicidios por grupos de edad. Comprobamos que tenemos las mismas columnas en ambas tablas. En esta (población por grupos de edad) confirmamos que son 2574, igual que en 'Defunciones por causa de muerte'.

In [ ]:
casi = casi.stack().reset_index()

In [ ]:
casi.columns = ['in_sexo', 'gr_edad', 'año', 'poblacion']

In [ ]:
casi['año'] = casi['año'].apply(lambda x: x.split()[1])

In [ ]:
casi

#### Exportamos la tabla para compararla con las defunciones por causa. 

In [ ]:
# casi.to_csv('poblacion_limpia_columnas.csv')